In [1]:
import cv2
import numpy as np
import os

from utils.visualization import *

In [2]:
def create_edge_mask(image, tongue_mask):
    height, width = image.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Find the contour of the tongue
    contours, _ = cv2.findContours(tongue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        tongue_contour = max(contours, key=cv2.contourArea)
        
        # Create a slightly smaller contour to focus on the edges
        epsilon = 0.02 * cv2.arcLength(tongue_contour, True)
        smaller_contour = cv2.approxPolyDP(tongue_contour, epsilon, True)
        
        # Draw the smaller contour on the mask
        cv2.drawContours(mask, [smaller_contour], 0, 255, thickness=int(width * 0.15))
    
    # crop top region
    mask[:int(height * 0.3), :] = 0  # Remove top 20%
    
    return mask

In [7]:
def extract_edges(image_path, image):
    _, tongue_mask = cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY)
    edge_mask = create_edge_mask(image, tongue_mask)

    # Apply the edge mask to the original image
    masked_image = cv2.bitwise_and(image, image, mask=edge_mask)
    vis_path = 'Data/extracted/test_visualization_toothmarks_' + image_path
    cv2.imwrite(vis_path, masked_image)

In [8]:
image_path = 'Data/to_extract'
images = os.listdir(image_path)
for image_file in images:
    image_path = 'Data/to_extract'
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_path, image_file)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to read image at {image_path}")
            continue
        
        extract_edges(image_file, image)